In [1]:
import openai
from langchain.llms import OpenAI
from langchain.chains import SequentialChain, SimpleSequentialChain, ConversationChain, LLMChain
from langchain.prompts import PromptTemplate
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.memory import ConversationBufferWindowMemory, ConversationBufferMemory

# other imports
from secret import API_KEY
# import os

#### Basic llms longchain

In [2]:
llm = OpenAI(openai_api_key = API_KEY,temperature=0.3)
answer = llm.predict("What is the basic of Programming?")
print(answer.strip())

The basic of programming is understanding how to write code that can be read and understood by a computer. This includes understanding the syntax of a programming language, as well as the logic and structure of the code. It also involves understanding how to debug and troubleshoot code, and how to optimize code for performance.


#### Simple Sequential chains

In [5]:
#chain1

template = """
        Write a short Answer the city
        {city}
        Write about the city """

prompt_city = PromptTemplate(template = template , input_variables = ["city"])

aboutcity = LLMChain(llm= llm, prompt=prompt_city)

city = input("Enter the Name of the city: ")

print(aboutcity.run(city))

Enter the Name of the city:  Delhi




Delhi is the capital of India and one of the most populous cities in the world. It is a vibrant and bustling metropolis, home to a diverse population of over 20 million people. Delhi is known for its rich cultural heritage, with monuments, temples, and markets that reflect its long history. It is also a major economic hub, with a thriving business district and a wide range of industries. Delhi is a great place to explore, with its bustling streets, vibrant nightlife, and delicious cuisine.


In [6]:
template = """
        Using the above information 
        {information}
        write a review about the city"""

prompt_review = PromptTemplate(template= template, input_variables=["information"])

aboutreview = LLMChain(llm= llm , prompt = prompt_review)
aboutreview_chain = SimpleSequentialChain(
    chains = [aboutcity, aboutreview]
)

aboutreview_chain.run(city)

" of Delhi\n\nDelhi is an amazing city with a rich history and culture. It is home to a variety of people from all over India and the world, and is a major hub for business, finance, and trade. The city is filled with a number of monuments, universities, museums, and other cultural attractions, making it a great destination for tourists. The iconic Taj Mahal, Humayun's Tomb, and the Lotus Temple are just a few of the many attractions that make Delhi a must-see destination. The city is vibrant and full of life, and is sure to leave a lasting impression on all who visit."

#### Simple Sequence Model


In [16]:

template = """
        Write a short Answer the city
        {city}
        Write about the city """

prompt_city = PromptTemplate(template = template , input_variables = ["city"])

aboutcity = LLMChain(llm= llm, prompt=prompt_city, output_key = "about city")

# city = input("Enter the Name of the city: ")

In [17]:
template = """
        Using the above information 
        {about city}
        write a review about the city"""

prompt_review = PromptTemplate(template= template, input_variables=["about city"])

aboutreview = LLMChain(llm= llm , prompt = prompt_review, output_key = "city review")

In [18]:
final_chain = SequentialChain(
    chains = [aboutcity, aboutreview],
    input_variables = ["city"],
    output_variables = ["about city", "city review"]
)

In [22]:
city = input("Enter the Name of the city: ")
text = final_chain({"city":city})

Enter the Name of the city:  Ranikhet


In [24]:
print(text["city"])

Ranikhet


In [25]:
print(text)

{'city': 'Ranikhet', 'about city': '\n\nRanikhet is a beautiful hill station located in the Almora district of Uttarakhand, India. It is situated at an altitude of 1869 meters above sea level and is surrounded by lush green forests and majestic snow-capped mountains. Ranikhet is known for its picturesque views, salubrious climate, and peaceful environment. It is a popular tourist destination and attracts a large number of visitors throughout the year. The town is home to several temples and other religious sites, as well as a number of historical monuments. The town is also known for its local handicrafts and delicious cuisine. Ranikhet is an ideal getaway for those who want to escape the hustle and bustle of city life and enjoy the tranquility of nature.', 'city review': '\n\nRanikhet is a beautiful hill station that offers a perfect getaway from the hustle and bustle of city life. The picturesque views, salubrious climate, and peaceful environment make it an ideal destination for tho

#### Agents

In [29]:
!pip install wikipedia

  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11685 sha256=bd1257eee3cda3df73aa76109a2c9f987bb7c3a542d2ab1e8ba9344d60643d0c
  Stored in directory: /home/deepak/.cache/pip/wheels/07/93/05/72c05349177dca2e0ba31a33ba4f7907606f7ddef303517c6a
Successfully built wikipedia


In [30]:
tools  = load_tools(['wikipedia', 'llm-math'], llm = llm)
agent = initialize_agent(
    tools, 
    llm,
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION
    
)

agent.run("What is the current GDP of India")

"India's nominal GDP in 2022 was estimated to be around 3.4% of the global economy."

### Memory 

In [41]:
memory = ConversationBufferMemory()
chain = LLMChain(llm = llm, prompt = PromptTemplate.from_template(
    "What is the fist {game} team to win world cup?",
      
    ),
    memory = memory,
)

In [42]:
chain.run("Cricket")

'\n\nThe first Cricket team to win the World Cup was the West Indies in 1975.'

In [43]:
chain.run("Footbal")

'\n\nThe first football team to win the World Cup was Uruguay in 1930.'

In [44]:
print(chain.memory)

chat_memory=ChatMessageHistory(messages=[HumanMessage(content='Cricket', additional_kwargs={}, example=False), AIMessage(content='\n\nThe first Cricket team to win the World Cup was the West Indies in 1975.', additional_kwargs={}, example=False), HumanMessage(content='Footbal', additional_kwargs={}, example=False), AIMessage(content='\n\nThe first football team to win the World Cup was Uruguay in 1930.', additional_kwargs={}, example=False)]) output_key=None input_key=None return_messages=False human_prefix='Human' ai_prefix='AI' memory_key='history'


In [46]:
print(chain.memory.buffer)

Human: Cricket
AI: 

The first Cricket team to win the World Cup was the West Indies in 1975.
Human: Footbal
AI: 

The first football team to win the World Cup was Uruguay in 1930.
